### Analisando perfil de transações que retornam chargeback

In [1]:
import pandas as pd
import numpy as np
import os, sys
import datetime as dt
from datetime import datetime

sys.path.insert(0, os.path.abspath(".."))
from src import utils as ult
from dotenv import load_dotenv ; load_dotenv()

# Importando plot functions
from src import plot_utils as plu

# Desligando warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Coleta o path_start, basicamente o local da pasta onde estão os dados.
path_start = os.getenv("PATH_START")
df = pd.read_csv(f"{path_start}data\\silver_dados_stone.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True) ; df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22948 entries, 0 to 22947
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Dia     22948 non-null  object 
 1   Hora    22948 non-null  object 
 2   Valor   22948 non-null  float64
 3   Cartão  22948 non-null  object 
 4   CBK     11128 non-null  object 
 5   label   22948 non-null  object 
dtypes: float64(1), object(5)
memory usage: 1.1+ MB


In [3]:
df['dia_hora'] = pd.to_datetime(df['Dia'] + ' ' + df['Hora'])
df['hour'] = df.apply(lambda x: x['dia_hora'].hour,axis=1)
df['shift'] = pd.cut(df['hour'], bins=[0, 7, 12, 18, 24], labels=["1", "2", "3", "4"], right=False)
df['week_number_month'] = df.apply(lambda x: (x['dia_hora'].day -1) // 7 + 1, axis=1)

df['weekday'] = df.apply(lambda x: x['dia_hora'].strftime("%A")[:3],axis=1)
weekday_mapping = {'Mon': 1, 
                    'Tue': 2, 
                    'Wed': 3, 
                    'Thu': 4, 
                    'Fri': 5, 
                    'Sat': 6, 
                    'Sun': 7}
df['weekday_num'] = df['weekday'].map(weekday_mapping)

df['shift'] = df['shift'].astype(int)
df['weekday_num'] = df['weekday_num'].astype(int)

In [46]:
def get_basic_statistics_customer(id_query: str, dia_hora: datetime, col: str, cbk):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora) & 
        (df['CBK'] == cbk)
    ]
    return df_check[col].mean()

def get_days_customer(id_query: str, dia_hora: datetime, cbk):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora) & 
        (df['CBK'] == cbk)
    ]
    if not df_check.empty:
        df_check['diff_days'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).days, axis=1)
        return df_check['diff_days'].mean()
    else:
        return -1

def get_last_day(id_query: str, dia_hora: datetime, cbk):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora) & 
        (df['CBK'] == cbk)
    ]
    if not df_check.empty:
        return (dia_hora - df_check['dia_hora'].max()).days
    
def get_days_operations(id_query: str, dia_hora, cbk: str, time_seconds: int):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora) & 
        (df['CBK'] == cbk)
    ]
    if not df_check.empty:
        df_check['diff_time'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).total_seconds() / 60, axis=1)
        return df_check[df_check['diff_time'] <= time_seconds].shape[0]
    else:
        return -1
    
def get_days_all_operations(id_query: str, dia_hora, time_seconds: int):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        df_check['diff_time'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).total_seconds() / 60, axis=1)
        return df_check[df_check['diff_time'] <= time_seconds].shape[0]
    else:
        return -1

def get_qtd_cbk_days(id_query: str, dia_hora, days: int):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora) & 
        (df['CBK'] == "Sim")
    ]
    if not df_check.empty:
        df_check['diff_time'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).days, axis=1)
        return df_check[df_check['diff_time'] <= days].shape[0]
    else:
        return -1

def get_qtd_ops(id_query: str, dia_hora, days: int):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        df_check['diff_time'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).days, axis=1)
        return df_check[df_check['diff_time'] <= days].shape[0]
    else:
        return -1

def get_mean_time_ops_all(id_query: str, dia_hora):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        df_check['diff_days'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).days, axis=1)
        return df_check['diff_days'].mean()
    else:
        return -1
   
def get_revenue_last_ops(id_query: str, dia_hora, days, type):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        df_check['diff_time'] = df_check.apply(lambda x: (dia_hora - x['dia_hora']).days, axis=1)
        if type == 'sum':
            return df_check[df_check['diff_time'] <= days]['Valor'].sum()
        else:
            return df_check[df_check['diff_time'] <= days]['Valor'].mean()
    else:
        return -1

def is_first_payment(id_query: str, dia_hora):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if df_check.empty:
        return 1
    else:
        return 0
    
def get_last_purchase_seconds(id_query: str, dia_hora):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        return (dia_hora - df_check['dia_hora'].max()).total_seconds()
    else:
        return -1
    
def is_last_payment_equal(id_query: str, dia_hora, payment):
    df_check = df[
        (df['Cartão'] == id_query) & 
        (df['dia_hora'] < dia_hora)
    ]
    if not df_check.empty:
        valor_ultima_compra = df_check.sort_values(by='dia_hora').iloc[-1]['Valor']
        return 1 if valor_ultima_compra == payment else 0
    else:
        return -1

In [7]:
df['is_first_payment'] = df.apply(lambda x: is_first_payment(
                                  id_query=x['Cartão'],
                                  dia_hora=x['dia_hora']),axis=1)

In [16]:
df['get_last_purchase_seconds'] = df.apply(lambda x: get_last_purchase_seconds(
                                            id_query=x['Cartão'],
                                            dia_hora=x['dia_hora']),axis=1)

In [47]:
df['is_last_payment_equal'] = df.apply(lambda x: is_last_payment_equal(
                                            id_query=x['Cartão'],
                                            dia_hora=x['dia_hora'],
                                            payment=x['Valor']),axis=1)

In [5]:
df['mean_shift_customer_no_cbk'] = df.apply(lambda x: get_basic_statistics_customer(
                                                                             id_query=x['Cartão'],
                                                                             dia_hora=x['dia_hora'],
                                                                             col='shift',
                                                                             cbk='Não'),axis=1)

df['mean_shift_customer_cbk'] = df.apply(lambda x: get_basic_statistics_customer(
                                                                             id_query=x['Cartão'],
                                                                             dia_hora=x['dia_hora'],
                                                                             col='shift',
                                                                             cbk='Sim'),axis=1)

In [7]:
df['mean_weekday_num_customer_no_cbk'] = df.apply(lambda x: get_basic_statistics_customer(
                                                                             id_query=x['Cartão'],
                                                                             dia_hora=x['dia_hora'],
                                                                             col='weekday_num',
                                                                             cbk='Não'),axis=1)

df['mean_weekday_num_customer_cbk'] = df.apply(lambda x: get_basic_statistics_customer(
                                                                             id_query=x['Cartão'],
                                                                             dia_hora=x['dia_hora'],
                                                                             col='weekday_num',
                                                                             cbk='Sim')
                                                        ,axis=1)

In [18]:
df[df['Cartão'] == '536518******2108']

,Dia,Hora,Valor,Cartão,CBK,label,dia_hora,hour,shift,week_number_month,weekday,weekday_num,is_first_payment,get_last_purchase_seconds
0,2015-05-01,00:01:54,36.54,536518******2108,Não,trainer,2015-05-01 00:01:54,0,1,1,Fri,5,1,-1.0
1,2015-05-01,00:03:46,36.54,536518******2108,Não,trainer,2015-05-01 00:03:46,0,1,1,Fri,5,0,112.0


In [16]:
df['mean_days_operations_customer_no_cbk'] = df.apply(lambda x: get_days_customer(
        id_query=x['Cartão'],
        dia_hora=x['dia_hora'],
        cbk='Não'
),axis=1)

df['mean_days_operations_customer_cbk'] = df.apply(lambda x: get_days_customer(
        id_query=x['Cartão'],
        dia_hora=x['dia_hora'],
        cbk='Sim'
),axis=1)

In [17]:
df[df['Cartão'] == '400225******8836']

,Dia,Hora,Valor,Cartão,CBK,dia_hora,hour,shift,week_number_month,weekday,weekday_num,mean_shift_customer_no_cbk,mean_shift_customer_cbk,mean_weekday_num_customer_no_cbk,mean_weekday_num_customer_cbk,mean_days_operations_customer_no_cbk,mean_days_operations_customer_cbk
8737,2015-05-25,16:14:02,291.2,400225******8836,Não,2015-05-25 16:14:02,16,3,4,Mon,1,NaN,NaN,NaN,NaN,-1.0,-1.0
9207,2015-05-27,18:12:09,112.0,400225******8836,Não,2015-05-27 18:12:09,18,4,4,Wed,3,3.0,NaN,1.0,NaN,2.0,-1.0


In [5]:
df['last_day_no_cbk'] = df.apply(lambda x: get_last_day(
        id_query=x['Cartão'],
        dia_hora=x['dia_hora'],
        cbk='Não'
),axis=1)

df['last_day_cbk'] = df.apply(lambda x: get_last_day(
        id_query=x['Cartão'],
        dia_hora=x['dia_hora'],
        cbk='Sim'
),axis=1)

In [6]:
df[df['Cartão'] == '400225******8836']

,Dia,Hora,Valor,Cartão,CBK,dia_hora,hour,shift,week_number_month,weekday,weekday_num,last_day_no_cbk,last_day_cbk
8737,2015-05-25,16:14:02,291.2,400225******8836,Não,2015-05-25 16:14:02,16,3,4,Mon,1,NaN,NaN
9207,2015-05-27,18:12:09,112.0,400225******8836,Não,2015-05-27 18:12:09,18,4,4,Wed,3,2.0,NaN


In [7]:
# get_days_operations

df['last_two_minutes_no_cbk_ops'] = df.apply(lambda x: get_days_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    cbk='Não',
    time_seconds=120
),axis=1)

df['last_two_minutes_cbk_ops'] = df.apply(lambda x: get_days_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    cbk='Sim',
    time_seconds=120
),axis=1)

df['last_five_minutes_no_cbk_ops'] = df.apply(lambda x: get_days_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    cbk='Não',
    time_seconds=300
),axis=1)

df['last_five_minutes_cbk_ops'] = df.apply(lambda x: get_days_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    cbk='Sim',
    time_seconds=300
),axis=1)

In [11]:
df['last_two_minutes_cbk_all_ops'] = df.apply(lambda x: get_days_all_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    time_seconds=120
),axis=1)

df['last_five_minutes_cbk_all_ops'] = df.apply(lambda x: get_days_all_operations(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    time_seconds=300
),axis=1)

In [12]:
df[df['Cartão'] == '400225******8836']

,Dia,Hora,Valor,Cartão,CBK,dia_hora,hour,shift,week_number_month,weekday,weekday_num,mean_shift_customer_no_cbk,mean_shift_customer_cbk,mean_weekday_num_customer_no_cbk,mean_weekday_num_customer_cbk,last_two_minutes_cbk_all_ops,last_five_minutes_cbk_all_ops
8737,2015-05-25,16:14:02,291.2,400225******8836,Não,2015-05-25 16:14:02,16,3,4,Mon,1,NaN,NaN,NaN,NaN,-1,-1
9207,2015-05-27,18:12:09,112.0,400225******8836,Não,2015-05-27 18:12:09,18,4,4,Wed,3,3.0,NaN,1.0,NaN,0,0


In [13]:
df['qtd_cbk_operations_15d'] = df.apply(lambda x: get_qtd_cbk_days(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    days=15
),axis=1)

df['qtd_all_operations_15d'] = df.apply(lambda x: get_qtd_ops(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    days=15
),axis=1)

df['mean_time_all_operations_15d'] = df.apply(lambda x: get_mean_time_ops_all(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora']
),axis=1)

df['mean_revenue_all_operations_15d'] = df.apply(lambda x: get_revenue_last_ops(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    days=15,
    type='mean'
),axis=1)

df['mean_revenue_all_operations_15d'] = df.apply(lambda x: get_revenue_last_ops(
    id_query=x['Cartão'],
    dia_hora=x['dia_hora'],
    days=15,
    type='sum'
),axis=1)

In [10]:
df[df['Cartão'] == '400225******8836']

,Dia,Hora,Valor,Cartão,CBK,dia_hora,hour,shift,week_number_month,weekday,...,last_day_no_cbk,last_day_cbk,last_two_minutes_no_cbk_ops,last_two_minutes_cbk_ops,last_five_minutes_no_cbk_ops,last_five_minutes_cbk_ops,qtd_cbk_operations_15d,qtd_all_operations_15d,mean_time_all_operations_15d,mean_revenue_all_operations_15d
8737,2015-05-25,16:14:02,291.2,400225******8836,Não,2015-05-25 16:14:02,16,3,4,Mon,...,NaN,NaN,-1,-1,-1,-1,-1,-1,-1.0,-1.0
9207,2015-05-27,18:12:09,112.0,400225******8836,Não,2015-05-27 18:12:09,18,4,4,Wed,...,2.0,NaN,0,-1,0,-1,-1,1,2.0,291.2


In [11]:
df['last_two_minutes_all_operations'] = df['last_two_minutes_no_cbk_ops'] + df['last_two_minutes_cbk_ops']
df['last_five_minutes_all_operations'] = df['last_five_minutes_no_cbk_ops'] + df['last_five_minutes_cbk_ops']
df['last_purchase_time'] = df.apply(lambda x: x['last_day_no_cbk'] if x['last_day_no_cbk'] <= x['last_day_cbk'] else x['last_day_cbk'], axis=1)

In [12]:
df[df['Cartão'] == '400225******8836']

,Dia,Hora,Valor,Cartão,CBK,dia_hora,hour,shift,week_number_month,weekday,...,last_two_minutes_cbk_ops,last_five_minutes_no_cbk_ops,last_five_minutes_cbk_ops,qtd_cbk_operations_15d,qtd_all_operations_15d,mean_time_all_operations_15d,mean_revenue_all_operations_15d,last_two_minutes_all_operations,last_five_minutes_all_operations,last_purchase_time
8737,2015-05-25,16:14:02,291.2,400225******8836,Não,2015-05-25 16:14:02,16,3,4,Mon,...,-1,-1,-1,-1,-1,-1.0,-1.0,-2,-2,NaN
9207,2015-05-27,18:12:09,112.0,400225******8836,Não,2015-05-27 18:12:09,18,4,4,Wed,...,-1,0,-1,-1,1,2.0,291.2,-1,-1,NaN
